In [229]:
#IMporting necessary packages 
import pandas as pd
from urllib.request import urlretrieve
import requests # used for http GET request
from bs4 import BeautifulSoup
import bs4 
import time
import re

In [55]:
# Importing data from indeed for chemical engineers
url1 = 'https://www.indeed.com/jobs?q=chemical+process+engineer'
che = requests.get(url1)
cheT = che.text
soup1 = BeautifulSoup(cheT, "html.parser")
print(BeautifulSoup.prettify(soup1))

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta content="text/html;charset=utf-8" http-equiv="content-type"/>
  <script src="/s/c524fdb/en_US.js" type="text/javascript">
  </script>
  <link href="/s/ecdfb5e/jobsearch_all.css" rel="stylesheet" type="text/css"/>
  <link href="http://rss.indeed.com/rss?q=chemical+process+engineer" rel="alternate" title="Chemical Process Engineer Jobs, Employment" type="application/rss+xml"/>
  <link href="/m/jobs?q=chemical+process+engineer" media="only screen and (max-width: 640px)" rel="alternate"/>
  <link href="/m/jobs?q=chemical+process+engineer" media="handheld" rel="alternate"/>
  <script type="text/javascript">
   if (typeof window['closureReadyCallbacks'] == 'undefined') {
        window['closureReadyCallbacks'] = [];
    }

    function call_when_jsall_loaded(cb) {
        if (window['closureReady']) {
            cb();
        } else {
            window['closureReadyCallbacks'].push(cb);
        }
    }
  </script>
  <meta content="1" name="p

In [52]:
# Extract Titles 

def extract_titles(soup):
    jobs = []
    for div in soup.find_all(name = 'div', attrs={'class':'row'}):
            for a in div.find_all(name = 'a', attrs= {'data-tn-element':'jobTitle'}):
                jobs.append(a['title'])
    return (jobs)

In [56]:
#Calling the function
extract_titles(soup1)

['Chemical Process Engineer',
 'Production Chemical Engineer',
 'Senior Process Engineer',
 'Plant Process Engineer',
 'Early Career Chemical Engineer – DuPont Engineering Rotational Program',
 'Process Engineer',
 'Process Engineer',
 'Process Engineer',
 'Production Engineer',
 'Product and Process Development Engineer',
 'Process Engineer (Greenville, SC)',
 'Chemical Engineer',
 'Chemical Engineer',
 'Process Engineer',
 'Chemical Engineer',
 'Process Improvement Engineer']

In [66]:
#Function to extract names

def extract_company_from_result(soup):
    companies = []
    for div in soup1.find_all(name='div', attrs={'class':'row'}):
        company = div.find_all(name='span', attrs={'class':'company'})
        if len(company) > 0:
            for b in company:
                companies.append(b.text.strip())
        else:
            sec_try = div.find_all(name='span', attrs={'class':'result-link-source'})
            for span in sec_try:
                companies.append(span.text.strip())
    return(companies)
 


In [67]:
extract_company_from_result(soup1)

['Weatherford Aerospace, LLC',
 'Engineering Executive Search Firm',
 'Fluid Quip Process Technologies, LLC',
 'Dart Container',
 'DuPont',
 'Placon Corporation',
 'Inteplast Group',
 'Flux Resources',
 'The Dow Chemical Company',
 'Total Quality Consulting',
 '3M',
 'YS, Inc.',
 'Redwood Materials, Inc.',
 'Mitsubishi Chemical Composites America - Quadrant',
 'Congoleum Corporation',
 'Huntsman']

In [71]:
# Function for location
def extract_location_from_result(soup):
    locations = []
    spans = soup.findAll('span', attrs={'class': 'location'})
    for span in spans:
        locations.append(span.text)
    return(locations)


In [72]:
extract_location_from_result(soup1)

['Weatherford, TX 76086',
 'La Porte, TX',
 'Cedar Rapids, IA',
 'Leola, PA 17540',
 'United States',
 'Madison, WI 53719',
 'Westborough, MA',
 'Hillsboro, OR',
 'Elizabethtown, KY',
 'Los Angeles, CA',
 'Greenville, SC',
 'Fort Payne, AL 35967',
 'Milpitas, CA 95035',
 'Scranton, PA',
 'Marcus Hook, PA 19061',
 'Port Neches, TX']

In [74]:
# Function for decription
def extract_summary_from_result(soup): 
    summaries = []
    spans = soup.findAll('span', attrs={'class': 'summary'})
    for span in spans:
        summaries.append(span.text.strip())
    return(summaries)


In [75]:
extract_summary_from_result(soup1)

['Chemical Process Engineer*. Chemical & Processes Engineer will be responsible for learning and executing the following tasks:....',
 'Production Chemical Engineer*. Bachelor’s degree in Chemical Engineering with 3 to 15 years’ experience working as a chemical engineer with relevant experience...',
 'Additional work requirements include the development of critical process flows, data tracking and the overall creation of a robust process engineering work flow...',
 'As a Plant Process Engineer you will be a representative of site compliance for OSHA Safety Management. Demonstrate knowledge of process equipment....',
 'There are multiple opportunities for engineers with a major in Chemical Engineering. Laboratory, pilot plant, or full-scale production tests are used by the...',
 'INDEED HIRE is partnering with Placon Corporation in search of dedicated and talented Process Engineers.*. Our Process Engineers are strong troubleshooters with...',
 'Responsible for providing detailed process

In [237]:
#Function for date
def extract_date_ago(soup):
    days = []
    spans = soup.findAll('span', attrs={'class':'date'})
    for span in spans:
        da = days.append(span.text.strip())
        re.findall('\d+',da)
    return (days)

In [238]:
extract_date_ago(soup)

TypeError: expected string or bytes-like object

In [247]:
# Filters
max_results_per_city = 100
city_set = ['Houston','Chicago','San+Francisco', 'Austin', 'Seattle', 'Los+Angeles', 'Philadelphia', 'Atlanta', 'Dallas', 'Pittsburgh', 'Portland', 'Denver', 'Virginia', 'Washington+DC', 'Boulder']
#city_set = ['Houston']
columns = ['city', 'job_title', 'company_name', 'location','days_ago','summary']
sample_df = pd.DataFrame(columns = columns)
print(sample_df)


Empty DataFrame
Columns: [city, job_title, company_name, location, days_ago, summary]
Index: []


In [248]:
#scraping code:
for city in city_set:
    for start in range(0, max_results_per_city, 10):
        page = requests.get('https://www.indeed.com/jobs?q=chemical+process+engineer&l=' + str(city) + '&start=' + str(start))
        time.sleep(1)  #ensuring at least 1 second between page grabs
        soup = BeautifulSoup(page.text, 'lxml')
        for div in soup.find_all(name='div', attrs={'class':'row'}): 
    #specifying row num for index of job posting in dataframe
            num = (len(sample_df) + 1) 
    #creating an empty list to hold the data for each posting
            job_post = [] 
    #append city name
            job_post.append(city) 
    #grabbing job title
            for a in div.find_all(name='a', attrs={'data-tn-element':'jobTitle'}):
                job_post.append(a['title']) 
    #grabbing company name
            company = div.find_all(name='span', attrs={'class':'company'})
            if len(company) > 0:
                for b in company:
                    job_post.append(b.text.strip()) 
            else:
                sec_try = div.find_all(name='span', attrs={'class':'result-link-source'})
                for span in sec_try:
                    job_post.append(span.text) 
    #grabbing location name
            c = div.findAll('span', attrs={'class': 'location'}) 
            for span in c:
                job_post.append(span.text) 
    #grabbing dates
            try:
                #e = div.findAll('span', attrs={'class': 'date'})
                #for span in e:
                    #job_post.append(span.text.strip())
                div_two = div.find(name='span', attrs={'class':'date'})
                job_post.append(div_two.text.strip())
            except:
                job_post.append('Nothing_found')
    #grabbing summary text
            d = div.findAll('span', attrs={'class': 'summary'})
            for span in d:
                job_post.append(span.text.strip())
            sample_df.loc[num] = job_post
            
    

In [249]:
#appending list of job post info to dataframe at index num
type(sample_df)
sample_df.head(1000)

,city,job_title,company_name,location,days_ago,summary
1,Houston,Production Chemical Engineer,Engineering Executive Search Firm,"La Porte, TX",Nothing_found,Production Chemical Engineer*. Bachelor’s degr...
2,Houston,Process Engineer-FTA000135,Pall,"Houston, TX 77040",Nothing_found,1+ years relevant work experience as process e...
3,Houston,Process Control Engineer,Paton Engineers and Constructors,"Houston, TX",Nothing_found,"Especially Process, Chemical and/or Computer E..."
4,Houston,Sr Process Engineer,W. R. Grace & Co.,"Pasadena, TX",Nothing_found,Previous experience as a Production or Process...
5,Houston,Senior Technical Professional - Process,KBR,"Houston, TX",11 days ago,Develop process flow diagrams (PFDs) and proce...
6,Houston,Process Engineer,ExxonMobil,"Houston, TX 77046 (Montrose area)",30+ days ago,"Process Engineer, Thermal Engineering, Enginee..."
7,Houston,Senior Process Technology Engineer,LyondellBasell Industries,"Houston, TX",5 days ago,The Senior Process Technology Engineer’s role ...
8,Houston,Process Engineer,Apex Life Sciences,"Houston, TX",29 days ago,Design and develop programs to facilitate new ...
9,Houston,Process Engineer,Solvay,"Deer Park, TX",29 days ago,Process Engineer - GBU005106. The specific dut...
10,Houston,Chemical Engineer,Offshore Commissioning Solutions Group,"Houston, TX",18 days ago,"Make use of process simulators, ChemCad, to ra..."


In [253]:
# Data Exploration
sample_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2263 entries, 1 to 2263
Data columns (total 6 columns):
city            2263 non-null object
job_title       2263 non-null object
company_name    2263 non-null object
location        2263 non-null object
days_ago        2263 non-null object
summary         2263 non-null object
dtypes: object(6)
memory usage: 203.8+ KB


In [255]:
sample_df.set_index('city')
sample_df.sort_index()

,city,job_title,company_name,location,days_ago,summary
1,Houston,Production Chemical Engineer,Engineering Executive Search Firm,"La Porte, TX",Nothing_found,Production Chemical Engineer*. Bachelor’s degr...
2,Houston,Process Engineer-FTA000135,Pall,"Houston, TX 77040",Nothing_found,1+ years relevant work experience as process e...
3,Houston,Process Control Engineer,Paton Engineers and Constructors,"Houston, TX",Nothing_found,"Especially Process, Chemical and/or Computer E..."
4,Houston,Sr Process Engineer,W. R. Grace & Co.,"Pasadena, TX",Nothing_found,Previous experience as a Production or Process...
5,Houston,Senior Technical Professional - Process,KBR,"Houston, TX",11 days ago,Develop process flow diagrams (PFDs) and proce...
6,Houston,Process Engineer,ExxonMobil,"Houston, TX 77046 (Montrose area)",30+ days ago,"Process Engineer, Thermal Engineering, Enginee..."
7,Houston,Senior Process Technology Engineer,LyondellBasell Industries,"Houston, TX",5 days ago,The Senior Process Technology Engineer’s role ...
8,Houston,Process Engineer,Apex Life Sciences,"Houston, TX",29 days ago,Design and develop programs to facilitate new ...
9,Houston,Process Engineer,Solvay,"Deer Park, TX",29 days ago,Process Engineer - GBU005106. The specific dut...
10,Houston,Chemical Engineer,Offshore Commissioning Solutions Group,"Houston, TX",18 days ago,"Make use of process simulators, ChemCad, to ra..."
